In [22]:
import xarray as xr
import pandas as pd
from prophet import Prophet
import cftime
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting

import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler


# Extracting data 

In [23]:
file_paths = ['CDS_Data/data_stream-wave_stepType-instant.nc', 'CDS_Data/data_stream-oper_stepType-instant.nc', 
           'CDS_Data/data_stream-oper_stepType-accum.nc']

# Create an empty list to store the Datasets
datasets = []

# Open each NetCDF file and append its Dataset to the list
for fp in file_paths:
    ds = xr.open_dataset(fp)
    datasets.append(ds)

# Merge all Datasets in the list into a single Dataset
combined_dataset = xr.merge(datasets)

# Inspect your dataset
print(combined_dataset)

<xarray.Dataset> Size: 17GB
Dimensions:     (valid_time: 365, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 3kB 2002-01-01T12:00:00 ... 2002-...
  * latitude    (latitude) float64 6kB -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B 0
    expver      (valid_time) <U4 6kB '0001' '0001' '0001' ... '0001' '0001'
Data variables:
    mwd         (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    mwp         (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    swh         (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    u10         (valid_time, latitude, longitude) float32 2GB ...
    v10         (valid_time, latitude, longitude) float32 2GB ...
    d2m         (valid_time, latitude, longitude) float32 2GB ...
    t2m         (valid_time, latitude, longitude) float32 2GB ...
    msl         (valid_time, latitud

In [24]:
print("--- Checking Latitude Coordinates ---")
print(combined_dataset['latitude'])
print(f"Minimum Latitude: {combined_dataset['latitude'].min().item()}")
print(f"Maximum Latitude: {combined_dataset['latitude'].max().item()}")

print("\n--- Checking Longitude Coordinates ---")
print(combined_dataset['longitude'])
print(f"Minimum Longitude: {combined_dataset['longitude'].min().item()}")
print(f"Maximum Longitude: {combined_dataset['longitude'].max().item()}")

print("\n--- Full Dataset Info ---")
print(combined_dataset) # Look closely at the dimensions here too.


--- Checking Latitude Coordinates ---
<xarray.DataArray 'latitude' (latitude: 721)> Size: 6kB
array([-90.  , -89.75, -89.5 , ...,  89.5 ,  89.75,  90.  ])
Coordinates:
  * latitude  (latitude) float64 6kB -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
    number    int64 8B 0
Attributes:
    units:             degrees_north
    standard_name:     latitude
    long_name:         latitude
    stored_direction:  decreasing
Minimum Latitude: -90.0
Maximum Latitude: 90.0

--- Checking Longitude Coordinates ---
<xarray.DataArray 'longitude' (longitude: 1440)> Size: 12kB
array([0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.5925e+02, 3.5950e+02,
       3.5975e+02])
Coordinates:
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number     int64 8B 0
Attributes:
    units:          degrees_east
    standard_name:  longitude
    long_name:      longitude
Minimum Longitude: 0.0
Maximum Longitude: 359.75

--- Full Dataset Info ---
<xarray.Dataset> Size: 17GB
Dimensions:     (

In [25]:
north_bound_lat_chch = 55.0  # Remember: -43.0 is "more north" than -44.0
south_bound_lat_chch = 45.0
west_bound_lon_chch = 5.0
east_bound_lon_chch = 20.0

# 1. Select the data for the specified Christchurch region
# IMPORTANT: For 'latitude' which is 'decreasing' (90 to -90), the slice order is (higher_value, lower_value)
ds = combined_dataset.sel(
    latitude=slice(south_bound_lat_chch, north_bound_lat_chch), # Correct order for decreasing latitude
    longitude=slice(west_bound_lon_chch, east_bound_lon_chch)   # Standard order for increasing longitude (0 to 359.5)
)

print(ds)

<xarray.Dataset> Size: 40MB
Dimensions:     (valid_time: 365, latitude: 41, longitude: 61)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 3kB 2002-01-01T12:00:00 ... 2002-...
  * latitude    (latitude) float64 328B 45.0 45.25 45.5 ... 54.5 54.75 55.0
  * longitude   (longitude) float64 488B 5.0 5.25 5.5 5.75 ... 19.5 19.75 20.0
    number      int64 8B 0
    expver      (valid_time) <U4 6kB '0001' '0001' '0001' ... '0001' '0001'
Data variables:
    mwd         (valid_time, latitude, longitude) float32 4MB nan nan ... 344.9
    mwp         (valid_time, latitude, longitude) float32 4MB nan nan ... 6.452
    swh         (valid_time, latitude, longitude) float32 4MB nan nan ... 2.092
    u10         (valid_time, latitude, longitude) float32 4MB ...
    v10         (valid_time, latitude, longitude) float32 4MB ...
    d2m         (valid_time, latitude, longitude) float32 4MB ...
    t2m         (valid_time, latitude, longitude) float32 4MB ...
    msl         (valid_time, latitude

In [26]:
df = ds.to_dataframe()
print(df)


                                        number expver         mwd       mwp  \
valid_time          latitude longitude                                        
2002-01-01 12:00:00 45.0     5.00            0   0001         NaN       NaN   
                             5.25            0   0001         NaN       NaN   
                             5.50            0   0001         NaN       NaN   
                             5.75            0   0001         NaN       NaN   
                             6.00            0   0001         NaN       NaN   
...                                        ...    ...         ...       ...   
2002-12-31 12:00:00 55.0     19.00           0   0001  351.784027  6.344384   
                             19.25           0   0001         NaN       NaN   
                             19.50           0   0001  347.205902  6.363671   
                             19.75           0   0001         NaN       NaN   
                             20.00           0   000

# Addressing NaNs in the data

In [27]:
print("\nNaN counts per column in DataFrame:")
print(df.isnull().sum())


NaN counts per column in DataFrame:
number         0
expver         0
mwd       884395
mwp       884395
swh       884395
u10            0
v10            0
d2m            0
t2m            0
msl            0
sst       821250
sp             0
tp             0
dtype: int64


In [28]:
# List of columns to drop
columns_to_drop = ['mwd', 'mwp', 'swh', 'sst']

# Drop the columns
df_cleaned = df.drop(columns=columns_to_drop)

print(print(df_cleaned.isnull().sum()))

number    0
expver    0
u10       0
v10       0
d2m       0
t2m       0
msl       0
sp        0
tp        0
dtype: int64
None


# Train/test split

In [29]:
df_reset = df_cleaned.reset_index()

# Convert 'valid_time' to datetime
df_reset['valid_time'] = pd.to_datetime(df_reset['valid_time'])

# Sort by time, then latitude, then longitude to ensure consistent grid formation
df_sorted = df_reset.sort_values(by=['valid_time', 'latitude', 'longitude'])

# Re-set the MultiIndex for easier handling, though we might iterate through it
df_final = df_sorted.set_index(['valid_time', 'latitude', 'longitude'])

print("DataFrame after sorting and re-indexing (first few rows):")
print(df_final.head())
print(f"Original shape: {df_final.shape}")



DataFrame after sorting and re-indexing (first few rows):
                                        number expver       u10       v10  \
valid_time          latitude longitude                                      
2002-01-01 12:00:00 45.0     5.00            0   0001 -0.804764 -1.102310   
                             5.25            0   0001 -0.738358 -0.229263   
                             5.50            0   0001 -0.763748  0.397690   
                             5.75            0   0001 -1.006912  0.909409   
                             6.00            0   0001 -1.389725  0.598862   

                                               d2m         t2m         msl  \
valid_time          latitude longitude                                       
2002-01-01 12:00:00 45.0     5.00       267.758881  274.061951  103693.125   
                             5.25       266.268646  273.729919  103658.125   
                             5.50       265.891693  271.888123  103644.625   
            

In [30]:
# --- 2. Determine Grid Dimensions ---
# Get unique latitudes and longitudes
latitudes = df_final.index.get_level_values('latitude').unique().sort_values()
longitudes = df_final.index.get_level_values('longitude').unique().sort_values()

num_lat = len(latitudes)
num_lon = len(longitudes)
num_features = len(df_final.columns) # All columns are features except if we designate one as target later

print(f"\nGrid dimensions: Latitude={num_lat}, Longitude={num_lon}")
print(f"Number of features (variables): {num_features}")



Grid dimensions: Latitude=41, Longitude=61
Number of features (variables): 9


In [31]:
# --- 3. Reshape to Gridded Data (Time, Lat, Lon, Features) ---
# Get all unique timestamps
all_times = df_final.index.get_level_values('valid_time').unique().sort_values()
num_times = len(all_times)

# Create an empty array to store the gridded data
# Shape: (num_times, num_lat, num_lon, num_features)
gridded_data = np.full((num_times, num_lat, num_lon, num_features), np.nan) # Use nan for missing spots

# Map latitude and longitude to array indices
lat_to_idx = {lat: i for i, lat in enumerate(latitudes)}
lon_to_idx = {lon: i for i, lon in enumerate(longitudes)}

# Populate the gridded_data array
for t_idx, time in enumerate(all_times):
    # Select data for the current timestamp
    time_slice_df = df_final.loc[time]

    for (lat, lon), row_data in time_slice_df.iterrows():
        lat_idx = lat_to_idx[lat]
        lon_idx = lon_to_idx[lon]
        gridded_data[t_idx, lat_idx, lon_idx, :] = row_data.values

print(f"\nShape of gridded_data: {gridded_data.shape}")
# Check if there are any NaNs left from this reshaping.
# There shouldn't be if all combinations of lat/lon exist for each time.
print(f"NaNs in gridded data after initial reshape: {np.sum(np.isnan(gridded_data))}")



Shape of gridded_data: (365, 41, 61, 9)
NaNs in gridded data after initial reshape: 0


In [32]:
# --- 4. Define Input (X) and Target (y) Variables ---
# 'tp' is total precipitation, which we want to predict.
# Let's find its column index.
feature_names = df_final.columns.tolist()
tp_col_idx = feature_names.index('tp')


In [33]:
# --- 5. Create Sequences for ConvLSTM ---
# We need to define:
# - `n_steps_in`: how many past time steps to use as input
# - `n_steps_out`: how many future time steps to predict (for rainfall, often 1)

n_steps_in = 5  # Example: Use the past 5 time steps (e.g., 5 days if data is daily)
n_steps_out = 1 # Example: Predict the next 1 time step (e.g., next day's rainfall)

def create_sequences(data, n_steps_in, n_steps_out, tp_idx):
    X, y = [], []
    for i in range(len(data) - n_steps_in - n_steps_out + 1):
        # Input sequence (all features except 'tp' for n_steps_in)
        # We need to exclude 'tp' from the input features
        X_seq_features = np.delete(data[i:(i + n_steps_in)], tp_idx, axis=-1)
        X.append(X_seq_features)

        # Output sequence (only 'tp' for n_steps_out)
        y_seq_tp = data[(i + n_steps_in):(i + n_steps_in + n_steps_out), :, :, tp_idx]
        y.append(y_seq_tp)
    return np.array(X), np.array(y)

X_sequences, y_sequences = create_sequences(gridded_data, n_steps_in, n_steps_out, tp_col_idx)

print(f"\nShape of X_sequences (input): {X_sequences.shape}")
print(f"Shape of y_sequences (target): {y_sequences.shape}")

# X_sequences shape should be (num_samples, n_steps_in, num_lat, num_lon, num_features - 1)
# y_sequences shape should be (num_samples, n_steps_out, num_lat, num_lon) - if predicting one variable




Shape of X_sequences (input): (360, 5, 41, 61, 8)
Shape of y_sequences (target): (360, 1, 41, 61)


In [34]:
# --- 6. Time-based Train-Test Split ---
# The data covers 2002. Let's use the first ~9 months for training and the last ~3 months for testing.
# A common split is 80% train / 20% test.
# Since X_sequences and y_sequences are already ordered by time, we can simply split the arrays.

# Determine the split point based on the number of sequences
split_ratio = 0.8
split_index = int(len(X_sequences) * split_ratio)

X_train = X_sequences[:split_index]
y_train = y_sequences[:split_index]
X_test = X_sequences[split_index:]
y_test = y_sequences[split_index:]

print(f"\nTraining set size: {len(X_train)} sequences")
print(f"Test set size: {len(X_test)} sequences")

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")



Training set size: 288 sequences
Test set size: 72 sequences
X_train shape: (288, 5, 41, 61, 8)
y_train shape: (288, 1, 41, 61)
X_test shape: (72, 5, 41, 61, 8)
y_test shape: (72, 1, 41, 61)


# Normalisation

In [35]:
original_feature_names_in_X_sequences = [col for col in df_final.columns if col != 'tp']


print(f"Features (channels) in X_sequences before normalization: {original_feature_names_in_X_sequences}")
print(f"Number of features (channels) in X_sequences: {len(original_feature_names_in_X_sequences)}")


Features (channels) in X_sequences before normalization: ['number', 'expver', 'u10', 'v10', 'd2m', 't2m', 'msl', 'sp']
Number of features (channels) in X_sequences: 8


In [36]:
# --- 1. Identify and Exclude Constant Features (if any) ---
# This step should ideally be done BEFORE creating X_sequences.
# Let's check for constants in the original `df_final` used to create the gridded data.
# If df_final is not available, you would need to load a sample or assume.

constant_features_found = []
if 'df_final' in locals():
    # Check only features that ended up in X_sequences
    for col in original_feature_names_in_X_sequences:
        if df_final[col].nunique() == 1:
            constant_features_found.append(col)
            print(f"Warning: Feature '{col}' is constant and will not be effectively normalized by RobustScaler.")
            print(f"It's generally recommended to remove constant features before creating sequences if they are truly constant.")
else:
    print("Warning: `df_final` not found. Cannot verify if features are constant.")


It's generally recommended to remove constant features before creating sequences if they are truly constant.
It's generally recommended to remove constant features before creating sequences if they are truly constant.


In [37]:
# --- 2. Normalization of X_train and X_test features ---

# Get the shape of X_train to reshape back later
shape_x_train = X_train.shape # (num_samples, n_steps_in, num_lat, num_lon, num_features_for_scaling)
num_features_for_scaling = shape_x_train[-1] # Number of channels (e.g., 8)

# Flatten X_train for scaler fitting: (total_elements, num_features)
# This combines all samples, timesteps, lat, and lon into a single dimension for feature scaling
X_train_reshaped_for_scaler = X_train.reshape(-1, num_features_for_scaling)
print(f"\nX_train reshaped for scaler fitting: {X_train_reshaped_for_scaler.shape}")

# Initialize RobustScaler for features
scaler_X = RobustScaler()

# Fit the scaler ONLY on the training data's reshaped features
scaler_X.fit(X_train_reshaped_for_scaler)
print("\nRobustScaler for features (X) fitted on X_train data.")

# Transform both training and test data using the fitted scaler
X_train_normalized_reshaped = scaler_X.transform(X_train_reshaped_for_scaler)
X_test_normalized_reshaped = scaler_X.transform(X_test.reshape(-1, num_features_for_scaling)) # Reshape X_test on the fly

print(f"X_train normalized reshaped shape: {X_train_normalized_reshaped.shape}")
print(f"X_test normalized reshaped shape: {X_test_normalized_reshaped.shape}")

# Reshape the normalized data back to the ConvLSTM input format (5D)
X_train_normalized = X_train_normalized_reshaped.reshape(shape_x_train)
X_test_normalized = X_test_normalized_reshaped.reshape(X_test.shape) # Use X_test's original shape

print(f"\nFinal X_train_normalized shape (ready for ConvLSTM input): {X_train_normalized.shape}")
print(f"Final X_test_normalized shape (ready for ConvLSTM input): {X_test_normalized.shape}")




X_train reshaped for scaler fitting: (3601440, 8)

RobustScaler for features (X) fitted on X_train data.
X_train normalized reshaped shape: (3601440, 8)
X_test normalized reshaped shape: (900360, 8)

Final X_train_normalized shape (ready for ConvLSTM input): (288, 5, 41, 61, 8)
Final X_test_normalized shape (ready for ConvLSTM input): (72, 5, 41, 61, 8)


In [38]:
# --- 3. Normalization for the Target Variable (y - Total Precipitation 'tp') ---
# 'tp' values are non-negative and often have a high concentration at zero, with some large values.
# RobustScaler can work, but consider alternatives like log1p if zeros are dominant and you want to reduce skewness.

# Get the shape of y_train to reshape back later
shape_y_train = y_train.shape # (num_samples, n_steps_out, num_lat, num_lon)
num_features_y = 1 # 'tp' is a single target variable

# Flatten y_train for scaler fitting: (total_elements, 1)
y_train_reshaped_for_scaler = y_train.reshape(-1, num_features_y)
print(f"\ny_train reshaped for scaler fitting: {y_train_reshaped_for_scaler.shape}")

# Initialize RobustScaler for the target 'tp'
scaler_y = RobustScaler()

# Fit the scaler ONLY on the training data's reshaped target
scaler_y.fit(y_train_reshaped_for_scaler)
print("RobustScaler for target (y/tp) fitted on y_train data.")

# Transform both training and test target data
y_train_normalized_reshaped = scaler_y.transform(y_train_reshaped_for_scaler)
y_test_normalized_reshaped = scaler_y.transform(y_test.reshape(-1, num_features_y))

y_train_normalized = y_train_normalized_reshaped.reshape(shape_y_train)
y_test_normalized = y_test_normalized_reshaped.reshape(y_test.shape)

print(f"Final y_train_normalized shape (ready for ConvLSTM output): {y_train_normalized.shape}")
print(f"Final y_test_normalized shape (ready for ConvLSTM output): {y_test_normalized.shape}")




y_train reshaped for scaler fitting: (720288, 1)
RobustScaler for target (y/tp) fitted on y_train data.
Final y_train_normalized shape (ready for ConvLSTM output): (288, 1, 41, 61)
Final y_test_normalized shape (ready for ConvLSTM output): (72, 1, 41, 61)


# Training

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Ensure TensorFlow is using a GPU if available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if len(tf.config.experimental.list_physical_devices('GPU')) > 0:
    print("Using GPU for training.")
else:
    print("No GPU found. Training on CPU.")

# --- Define Model Parameters based on your data ---
# These values should come from your previous data preparation steps
# For demonstration purposes, I'll use placeholders if not defined globally.
# In a continuous script, these would already be in scope.

# Example values (replace with your actual data shapes from previous steps)
# from the previous notebook section
# X_train_normalized shape: (num_train_samples, n_steps_in, num_lat, num_lon, num_predictor_features)
# y_train_normalized shape: (num_train_samples, n_steps_out, num_lat, num_lon)

# Assume these are available from the previous run:
# X_train_normalized, y_train_normalized, X_test_normalized, y_test_normalized

# Infer dimensions from your normalized data
n_samples_train, n_steps_in, num_lat, num_lon, num_channels = X_train_normalized.shape
_, n_steps_out, _, _ = y_train_normalized.shape # Assuming y_train_normalized has 4 dimensions

print(f"\nModel Input Parameters:")
print(f"  n_steps_in (sequence length): {n_steps_in}")
print(f"  num_lat (grid rows): {num_lat}")
print(f"  num_lon (grid columns): {num_lon}")
print(f"  num_channels (input features): {num_channels}")
print(f"  n_steps_out (prediction horizon): {n_steps_out}")

# --- Adjust y_train_normalized/y_test_normalized if n_steps_out is 1 ---
# Keras ConvLSTM2D output can be a sequence or just the last step.
# If n_steps_out is 1, it's common to predict a single frame.
# The target `y` should match the model's output shape.
if n_steps_out == 1:
    # Squeeze the n_steps_out dimension from y if it's 1, as the model's Conv3D output will be 3D.
    # From (samples, 1, lat, lon) to (samples, lat, lon)
    y_train_target = y_train_normalized.squeeze(axis=1)
    y_test_target = y_test_normalized.squeeze(axis=1)
    print(f"Target 'y' adjusted from {y_train_normalized.shape} to {y_train_target.shape} for n_steps_out=1.")
else:
    y_train_target = y_train_normalized
    y_test_target = y_test_normalized
    print(f"Target 'y' shape remains {y_train_target.shape} as n_steps_out > 1.")


# --- Build the ConvLSTM Model ---

model = Sequential()

# Layer 1: ConvLSTM2D
# filters: Number of convolutional filters. More filters can capture more patterns.
# kernel_size: The dimensions of the convolution window (e.g., (3,3)).
# activation: Activation function. 'relu' is common.
# padding: 'same' to maintain spatial dimensions, 'valid' to reduce.
# return_sequences: True to stack another ConvLSTM2D layer, False for the last ConvLSTM2D.
# input_shape: (n_steps_in, num_lat, num_lon, num_channels)
model.add(ConvLSTM2D(filters=64, kernel_size=(5, 5),
                     activation='relu',
                     padding='same',
                     return_sequences=True, # Return sequences if you plan to stack more ConvLSTM2D
                     input_shape=(n_steps_in, num_lat, num_lon, num_channels)))
model.add(BatchNormalization())

# Layer 2: ConvLSTM2D (Optional - you can add more or fewer)
model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     activation='relu',
                     padding='same',
                     return_sequences=False)) # Set to False for the last ConvLSTM2D to output a single sequence item
model.add(BatchNormalization())

# Output Layer: Conv3D for predicting a sequence of 2D grids OR Conv2D if n_steps_out=1
# Since we designed for n_steps_out = 1 and squeezed y, we need an output that is (lat, lon)
# After the last ConvLSTM2D (return_sequences=False), the output is (batch_size, num_lat, num_lon, filters)
# We need to transform this to (batch_size, num_lat, num_lon) for a single precipitation value.

# A Conv2D layer can project the `filters` down to 1 channel (tp).
# The input to this Conv2D will be (batch_size, num_lat, num_lon, last_convlstm_filters)
# The output will be (batch_size, num_lat, num_lon, 1)
# Then squeeze the last dimension.

# An alternative for predicting a single 2D grid:
# Using Conv3D to predict a single frame (n_steps_out = 1).
# The output of the last ConvLSTM2D is (batch_size, num_lat, num_lon, filters).
# We need to add a dimension for time step 1 to use Conv3D, or simply use Conv2D.
# Let's use Conv2D to output the single frame prediction.

# Conv2D output layer: 1 filter (for 'tp'), 1x1 kernel (to combine feature maps), linear activation.
# This projects the feature maps from the last ConvLSTM2D into a single precipitation map.
# If you want to predict n_steps_out > 1, you'd wrap this in a TimeDistributed layer
# and use Conv3D with 1 filter for the final step.
# For n_steps_out = 1, a simple Conv2D is appropriate to output the 2D grid.
model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), activation='linear', padding='same')) # Outputting a 2D grid (lat, lon)

# Squeeze the last dimension if it's 1 (from (batch, lat, lon, 1) to (batch, lat, lon))
# Keras models automatically handle this if the loss function expects a different shape,
# but it's good practice to ensure the model's output matches the target's shape.
# We'll use a Lambda layer for this if n_steps_out is 1.
if n_steps_out == 1:
    # Ensure the model's output directly matches y_train_target's shape (batch, lat, lon)
    model.add(tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=-1)))


# --- Compile the Model ---
optimizer = Adam(learning_rate=0.001) # You can adjust learning rate
model.compile(optimizer=optimizer, loss='mse', metrics=['mae']) # Mean Absolute Error (MAE) is good for interpretability

model.summary()

# --- Define Callbacks for Training ---
# EarlyStopping: Stop training if validation loss doesn't improve
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ModelCheckpoint: Save the best model based on validation loss
model_checkpoint = ModelCheckpoint('best_convlstm_model.h5',
                                   monitor='val_loss',
                                   save_best_only=True,
                                   mode='min',
                                   verbose=1)

# --- Train the Model ---
print("\nStarting model training...")
history = model.fit(X_train_normalized, y_train_target,
                    epochs=50, # Number of epochs, adjust as needed
                    batch_size=8, # Batch size, adjust based on GPU memory
                    validation_split=0.2, # Use a portion of training data for validation
                    callbacks=[early_stopping, model_checkpoint])

print("\nModel training complete.")

# --- Evaluate the Model on the Test Set ---
print("\nEvaluating model on the test set...")
test_loss, test_mae = model.evaluate(X_test_normalized, y_test_target, verbose=1)
print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")

# You can now use the trained `model` to make predictions:
# predictions_normalized = model.predict(X_test_normalized)
# Then inverse transform `predictions_normalized` using `scaler_y` to get original precipitation values.

Num GPUs Available:  0
No GPU found. Training on CPU.


NameError: name 'X_train_normalized' is not defined

# Prediction